# Estudo de caso: Preferências de drinks

O conjunto de dados deste estudo de caso vem de uma pesquisa realizada com adultos maiores de 18 anos. O estudo contém medidas biológicas (idade, peso, altura) e preferências de drinks (alcoólicos e não-alcoólicos). O objetivo é realizar o pré-processamento dos dados e finalizar com uma base limpa e preparada para análise em si.

As colunas são:

- `'resp_id'` (número de identificação que identifica cada pessoa de forma única)

- `'age'` (idade)

- `'gender'` ('Male' ou 'Female')

- `'height'` (em polegadas)

- `'weight'` (em libras)

- `'nonalcoholic_drink'` (bebida não-alcoólica favorita)

- `'alcoholic_drink'` (bebida alcoólica favorita)

Vamos lá!

In [1]:
#Importando a biblioteca Pandas
import pandas as pd

#Lendo a base de dados
drink_preferences = pd.read_csv('/datasets/drink_preferences.csv')

#Imprimindo as 10 primeiras linhas da base de dados
print(drink_preferences.head(10))

#Criando um espaço entre as informações
print()

#Gerando as informações gerais dos dados
drink_preferences.info()

   resp_id   age  gender  height  weight nonalcoholic_drink alcoholic_drink
0     1001  19.0    Male     NaN     NaN             Coffee             NaN
1     1002   NaN  Female   64.50  128.70              Water          Liquor
2     1003  21.0    Male   71.47  182.95             Coffee            Beer
3     1004  32.0  Female   57.30  103.40          Green Tea            Wine
4     1005  32.0  Female   53.80  138.40          Black Tea          Liquor
5     1006  20.0    Male   73.38  204.59              Pepsi             NaN
6     1007  20.0    Male   70.46  225.25             Coffee             NaN
7     1008  32.0  Female   54.10  157.80          Black Tea          Liquor
8     1009  22.0    Male     NaN  197.00             Coffee          Liquor
9     1009  22.0    Male     NaN  197.00             Coffee          Liquor

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  -----

## Pré-processamento de dados

Observando os dados, nota-se inicialmente que há tanto valores ausentes quanto valores duplicados. 

Os valores ausentes são notáveis pela diferença de valores não-nulos nas informações da base de dados, já quanto aos valores duplicados, nota-se que as linhas de índice 8 e 9 estão duplicadas.

Iniciaremos com o tratamento dos dados duplicados.

### Valores duplicados 

Primeiro, analisaremos os valores duplicados e qual será a melhor forma de tratamento.


In [2]:
# Analisando as duplicatas a partir do filtro de números de identidade duplicados
drink_preferences[drink_preferences.duplicated(subset= 'resp_id')]

,resp_id,age,gender,height,weight,nonalcoholic_drink,alcoholic_drink
9,1009,22.0,Male,NaN,197.00,Coffee,Liquor
102,1101,29.0,Female,60.80,137.30,Pepsi,None
237,1238,33.0,Male,66.18,197.34,Apple Juice,NaN


Com esse primeiro código, notamos que há três entrevistados que aparecem com as informações duplicadas na base de dados. Dessa forma, é hora de começar o tratamento dos valores.

In [3]:
# Testando o primeiro método: drop_duplicates()
drink_preferences= drink_preferences.drop_duplicates()

# Esse código vai verificar o resultado
drink_preferences[drink_preferences.duplicated(subset='resp_id')]

,resp_id,age,gender,height,weight,nonalcoholic_drink,alcoholic_drink
237,1238,33.0,Male,66.18,197.34,Apple Juice,NaN


Hum, isso é interessante! 
Retiramos as duplicatas com o método drop_duplicates(), porém, ainda sobrou uma. Isso significa que é necessário um maior cuidado com essa linha de dados, que deve ser analisada separadamente.

In [4]:
#Analisando a linha duplicada remanescente
drink_preferences[drink_preferences['resp_id'] == 1238]

,resp_id,age,gender,height,weight,nonalcoholic_drink,alcoholic_drink
236,1238,33.0,Male,66.18,197.34,NaN,Liquor
237,1238,33.0,Male,66.18,197.34,Apple Juice,NaN


Com o código anterior, foi possível notar que as linhas de índice 236 e 237 não são exatamente iguais. 

Enquanto a primeira possui o valor `nonalcoholic_drink` como Nulo, a segunda apresenta como nulo o valor de `alcoholic_drink`. 

Os dados, dessa forma, devem ter sido divididos em duas linhas durante a exportação do conjunto de dados.

Agora que sabemos a razão dessa divergência, podemos condensar essas informações em apenas uma linha, retirando efetivamente, dessa forma, todas as duplicatas da base de dados.

In [5]:
# Preenchendo os valores nulos com as informações de ambas as linhas com o método .loc()
drink_preferences.loc[236,'nonalcoholic_drink'] = 'Apple Juice'
drink_preferences.loc[237,'alcoholic_drink'] = 'Liquor'

#Excluindo todas as linhas duplicadas
drink_preferences= drink_preferences.drop_duplicates()

# Esse código vai verificar o resultado
print(drink_preferences[drink_preferences['resp_id'] == 1238])
print()
drink_preferences[drink_preferences.duplicated(subset='resp_id')]

     resp_id   age gender  height  weight nonalcoholic_drink alcoholic_drink
236     1238  33.0   Male   66.18  197.34        Apple Juice          Liquor



,resp_id,age,gender,height,weight,nonalcoholic_drink,alcoholic_drink


Pronto! Agora a linha 236 é a única com o id 1238, assim como o DataFrame que retornaria as duplicatas está vazio! 

É hora de passar para os valores ausentes.

### Valores ausentes

Importante: O índice foi resetado e os dados limpos sem duplicatas foram salvos em um novo arquivo chamado `drink_pref_cleaned.csv`

Algumas hipóteses quanto à razão da existência dos valores ausentes são:
- Perguntas não respondidas pelos entrevistados
- Entrevistado menor de 21 anos (onde 21 = idade mínima para bebidas alcoólicas), não podendo responder sobre sua preferência para bebidas alcoólicas
- Erro na construção/exportação da base de dados

In [6]:
#Verificando os valores únicos em que está preenchida a coluna 'alcoholic_drink' para menores de 21 anos
print(drink_preferences.loc[drink_preferences['age'] < 21]['alcoholic_drink'].unique())

[nan]


Existe apenas um valor único para entrevistados menores de 21 anos: NaN. Nossa teoria estava certa!

Nós temos três tipos de valores ausentes para considerar para bebida alcoólica preferida:

- Não há respostas porque o entrevistado tinha menos de 21 anos

- O entrevistado tem 21 anos ou mais, mas não deu resposta por alguma razão desconhecida

- A idade do entrevistado também está ausente

Vamos usar dois valores representativos diferentes para preencher os dados ausentes: `'N/A'` para entrevistados com menos 21 e `'Unknown'` para entrevistados que têm 21 anos ou mais, ou entrevistados que não preencheram o valor da idade.

In [7]:
#Preenchendo valores ausentes com 'N/A' e 'Unknown'
drink_preferences.loc[drink_preferences['age'] < 21, 'alcoholic_drink'] = 'N/A'
drink_preferences['alcoholic_drink'] = drink_preferences['alcoholic_drink'].fillna(value='Unknown')

# Esse código vai verificar o resultado
drink_preferences.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 0 to 299
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   resp_id             297 non-null    int64  
 1   age                 261 non-null    float64
 2   gender              297 non-null    object 
 3   height              278 non-null    float64
 4   weight              282 non-null    float64
 5   nonalcoholic_drink  297 non-null    object 
 6   alcoholic_drink     297 non-null    object 
dtypes: float64(3), int64(1), object(3)
memory usage: 18.6+ KB


Essas são 297 linhas e 297 valores não-nulos na coluna 'alcoholic_drink'. Agora, apenas nossas colunas numéricas tem valores ausentes, então vamos cuidar deles.

Vamos começar contando o número de valores numéricos ausentes para cada gênero. 

In [8]:
#Criando uma lista com as colunas de dados numéricos
numeric_columns = ['age', 'height', 'weight']

#Criando uma lista vazia para inserir os valores faltantes
missing_values = []

#Criando um ciclo for para 1) Filtrar os valores ausentes nas colunas numéricas e adicionar à lista missing_values os valores filtrados por gênero:
for column in numeric_columns:
    filtered_data = drink_preferences[drink_preferences[column].isna()]
    missing_values.append(filtered_data['gender'].value_counts())

# Esse código vai verificar o resultado
print('Count of missing age values by gender')
print(missing_values[0])
print()
print('Count of missing height values by gender')
print(missing_values[1])
print()
print('Count of missing weight values by gender')
print(missing_values[2])

Count of missing age values by gender
Female    20
Male      16
Name: gender, dtype: int64

Count of missing height values by gender
Male    19
Name: gender, dtype: int64

Count of missing weight values by gender
Male    15
Name: gender, dtype: int64


Diante desse resultado, é possível observar que temos valores ausentes para ambos os gêneros, porém, os valores de altura e peso estão ausentes apenas para os homens. Dessa forma, vamos calcular a média e mediana sobre os valores ausentes, para poder substituí-los mais tarde.

In [9]:
# Lista com as colunas de dados numéricos
numeric_columns = ['age', 'height', 'weight']

#Calculando a média dos valores de idade, altura e peso
mean_values= drink_preferences.groupby('gender')[numeric_columns].mean()

#Calculando a mediana dos valores de idade, altura e peso
median_values= drink_preferences.groupby('gender')[numeric_columns].median()

# Esse código vai verificar o resultado
print('Essas são as médias:')
print(mean_values)
print()
print('Essas são as medianas:')
print(median_values)

These are the means:
              age     height      weight
gender                                  
Female  27.793893  59.734437  148.594040
Male    25.746154  69.945354  209.346031

These are the medians:
         age  height  weight
gender                      
Female  23.0   59.80  148.40
Male    22.0   69.73  209.77


Analisando os dados médios e medianos, notamos que:
- A média de altura dos homens no nosso conjunto de dados é mais ou menos 25 cm mais alto do que as mulheres, e a média de peso mais ou menos 18,1 kg mais pesado! 
- Vamos usar a média masculina para preencher os valores ausentes de altura e peso.
- Idade é mais ou menos a mesma para ambos os gêneros, então usaremos a mediana geral de idade para preencher os valores ausentes.

In [10]:
# Calculando os valores substitutos (respectivamente, mediana das idades e média da altura e peso masculino)
median_age = drink_preferences['age'].median()
male_mean_height = drink_preferences[drink_preferences['gender'] == 'Male']['height'].mean()
male_mean_weight = drink_preferences[drink_preferences['gender'] == 'Male']['weight'].mean()

# Preenchendo os valores ausentes
drink_preferences['age'] = drink_preferences['age'].fillna(median_age)
drink_preferences['height'] = drink_preferences['height'].fillna(male_mean_height)
drink_preferences['weight'] = drink_preferences['weight'].fillna(male_mean_weight)

# Verificando as informações não nulas da tabela para verificar o resultado
drink_preferences.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 0 to 299
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   resp_id             297 non-null    int64  
 1   age                 297 non-null    float64
 2   gender              297 non-null    object 
 3   height              297 non-null    float64
 4   weight              297 non-null    float64
 5   nonalcoholic_drink  297 non-null    object 
 6   alcoholic_drink     297 non-null    object 
dtypes: float64(3), int64(1), object(3)
memory usage: 18.6+ KB


Prontinho! Tanto os valores ausentes quanto duplicados foram tratados!

# Conclusões

O conjunto de dados recebido estava cheio de duplicatas e dados ausentes. 

Neste estudo de caso, encontrei identidades de entrevistados duplicadas e os removi de acordo com o tipo de duplicatas que eles eram. 

Também identifiquei valores ausentes categóricos e quantitativos, e examinei a relação com os dados para preencher esses valores.

Por fim, o conjunto de dados foi tratado e está pronto para análise.